In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import utils
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [23]:
events = utils.get_events()
events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,month,day,hora
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0


In [24]:
campaign_events = events[events['event']=='ad campaign hit'][['timestamp', 'event', 'person', 'url',
       'campaign_source', 'day', 'month']]
campaign_events['ads_accion'] = campaign_events['url'].str.split('/').str[1]
campaign_events['ads_marca'] = campaign_events['url'].str.split('/').str[2]
campaign_events['ads_modelo'] = campaign_events['url'].str.split('/').str[3]

In [33]:
## source
campaign_data = campaign_events[campaign_events['campaign_source'].notnull()]
campaign_data['campaign_source'].value_counts()

google              123354
criteo               24931
rtbhouse             23540
zanox                 7013
emblue                3653
afilio                1913
manifest              1365
FacebookAds           1301
voxus                  971
rakuten                843
bing                   839
datacrush              433
indexa                 313
buscape                271
Facebook               203
afiliado                92
blog                    88
mercadopago             74
yotpo                   54
FacebookSocial          12
onsite                  12
MARKETING SOCIAL         7
gizmodo                  4
Name: campaign_source, dtype: int64

In [34]:
campaign_data = campaign_data[['person', 'campaign_source', 'timestamp']].pivot_table(index='person', columns='campaign_source', values='timestamp', aggfunc='count', fill_value=0)
campaign_data.columns = campaign_data.columns.astype('object')
campaign_data.reset_index(inplace=True)
campaign_data.columns = ['campaign-source_'+col if col != 'person' else col for col in campaign_data.columns]
data = campaign_data.fillna(0)
campaign_data.shape

(31887, 24)

In [35]:
## marca
campaign_data = campaign_events[campaign_events['ads_marca'].notnull()]
campaign_data['ads_marca'].value_counts()

samsung     63029
iphone      43578
motorola    14899
lg           2658
sony         1073
asus          505
lenovo        466
              326
ipad          274
quantum        44
ofertas        26
list           24
apple          17
nokia          15
Name: ads_marca, dtype: int64

In [36]:
campaign_data = campaign_events[['person', 'ads_marca', 'timestamp']].pivot_table(index='person', columns='ads_marca', values='timestamp', aggfunc='count', fill_value=0)
campaign_data.columns = campaign_data.columns.astype('object')
campaign_data.reset_index(inplace=True)
campaign_data.columns = ['campaign-marca_'+col if col != 'person' else col for col in campaign_data.columns]
data  = pd.merge(data, campaign_data, on='person', how='left').fillna(0)
data.head()

,person,campaign-source_Facebook,campaign-source_FacebookAds,campaign-source_FacebookSocial,campaign-source_MARKETING SOCIAL,campaign-source_afiliado,campaign-source_afilio,campaign-source_bing,campaign-source_blog,campaign-source_buscape,...,campaign-marca_iphone,campaign-marca_lenovo,campaign-marca_lg,campaign-marca_list,campaign-marca_motorola,campaign-marca_nokia,campaign-marca_ofertas,campaign-marca_quantum,campaign-marca_samsung,campaign-marca_sony
0,00091926,0,0,0,0,0,0,0,0,0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0
1,00091a7a,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,000ba417,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,000c79fe,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,000e4d9e,0,0,0,0,0,4,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0


In [37]:
data.shape

(31887, 38)

In [38]:
data.to_csv('features_ad_campaing.csv',index=False)